# Set-up

In [1]:
#pip install dash

In [13]:
import pandas as pd
import re
import sklearn
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
import random
random.seed(42)

import dash
from dash import dcc, html
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
from plotly.offline import iplot, init_notebook_mode

In [3]:
data_home = "C:/Users/johnw/Box/Michigan Parent Dashboard/Michigan Data"

MIGENEDUSCHOOLDATA = pd.read_csv(f"{data_home}/MIGENEDUSCHOOLDATA_11_27_23.csv", low_memory=False)

In [4]:
MIGENEDCOMPLETEWITHECA = MIGENEDUSCHOOLDATA[MIGENEDUSCHOOLDATA['extracurricular_missing'] == 0]

# Create new columns with initial values as 0
MIGENEDCOMPLETEWITHECA['EC_Sports'] = 0
MIGENEDCOMPLETEWITHECA['EC_Academics'] = 0
MIGENEDCOMPLETEWITHECA['EC_Arts'] = 0
MIGENEDCOMPLETEWITHECA['EC_Civics'] = 0
MIGENEDCOMPLETEWITHECA['EC_Vocational'] = 0

# Set to 1 if School has a particular type of extracurricular
MIGENEDCOMPLETEWITHECA.loc[(MIGENEDCOMPLETEWITHECA['EC_SPORTS_BASEBALL'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_SPORTS_BSKTBALL'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_SPORTS_BOWLING'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_SPORTS_DIVING'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_SPORTS_COMPCHEER'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_SPORTS_CROSSCNTRY'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_SPORTS_FOOTBALL'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_SPORTS_GOLF'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_SPORTS_GYMNAST'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_SPORTS_ICEHOCKEY'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_SPORTS_OTHSPRTS'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_SPORTS_POWERLIFT'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_SPORTS_SKIING'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_SPORTS_SOFTBALL'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_SPORTS_SOCCER'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_SPORTS_SWIM'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_SPORTS_TENNIS'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_SPORTS_TRCKFIELD'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_SPORTS_VOLBALL'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_SPORTS_WRSTL'] == 1), 'EC_Sports'] = 1
MIGENEDCOMPLETEWITHECA.loc[(MIGENEDCOMPLETEWITHECA['EC_CLUBS_HONSOC'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_CLUBS_LITHUM'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_CLUBS_ROBOTICS'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_CLUBS_STEM'] == 1), 'EC_Academics'] = 1
MIGENEDCOMPLETEWITHECA.loc[(MIGENEDCOMPLETEWITHECA['EC_CLUBS_MUSIC'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_CLUBS_VISARTS'] == 1), 'EC_Arts'] = 1
MIGENEDCOMPLETEWITHECA.loc[(MIGENEDCOMPLETEWITHECA['EC_CLUBS_NSLEAD'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_CLUBS_PEERTUTMEN'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_CLUBS_POLGOV'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_CLUBS_SCHOOLCOM'] == 1) | (MIGENEDCOMPLETEWITHECA['EC_CLUBS_SPCHDEB'] == 1), 'EC_Civics'] = 1
MIGENEDCOMPLETEWITHECA.loc[(MIGENEDCOMPLETEWITHECA['EC_CLUBS_CARFOC'] == 1) , 'EC_Vocational'] = 1

C:\Users\johnw\AppData\Local\Temp\ipykernel_27512\397745033.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\johnw\AppData\Local\Temp\ipykernel_27512\397745033.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\johnw\AppData\Local\Temp\ipykernel_27512\397745033.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [5]:
# Initialize Plotly in offline mode for Jupyter Notebook
init_notebook_mode(connected=True)

# Quantity of Extracurricular Activities Available and Demographics

In [24]:
# Create a Dash application
app = dash.Dash(__name__)

# Define school levels and x-axis options
school_levels = ['Elementary School', 'Middle School', 'High School']
x_axis_options = ['per_white', 'per_asian', 'per_africanamerican', 'per_hispanic', 'per_other',
                  'per_whiteasian', 'per_blackorhisp'
]

# Define corresponding labels for the x-axis options
x_axis_labels = [
    'White', 'Asian', 'Black', 'Hispanic', 'Other',
    'White & Asian', 'Black & Hispanic'
]

# Combine options with labels for the dropdown
x_axis_dropdown_options = [{'label': label, 'value': value} for label, value in zip(x_axis_labels, x_axis_options)]

# Define a dictionary to map options to updated labels
updated_labels = {
    'per_whiteasian': 'White and Asian',
    'per_blackorhisp': 'Black and Hispanic'
}

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Quantity of Extracurricular Activities Available"),
    dcc.Dropdown(
        id='outcome-dropdown',
        options=[
            {'label': 'Number of Extracurriculars', 'value': 'numextracurricular'},
            {'label': 'Number of Clubs', 'value': 'numclubs'},
            {'label': 'Number of Sports', 'value': 'numsports'}
        ],
        value='numextracurricular'  # Default value
    ),
    dcc.Dropdown(
        id='school-level-dropdown',
        options=[{'label': level, 'value': level} for level in school_levels],
        value='Elementary School'  # Default value
    ),
    dcc.Dropdown(
        id='x-axis-dropdown',
        options=x_axis_dropdown_options,
        value='per_white'  # Default value
    ),
    dcc.Graph(id='scatter-plot')
])

@app.callback(
    dash.dependencies.Output('scatter-plot', 'figure'),
    [dash.dependencies.Input('school-level-dropdown', 'value'),
     dash.dependencies.Input('x-axis-dropdown', 'value'),
     dash.dependencies.Input('outcome-dropdown', 'value')]
)
def update_scatter_plot(selected_school_level, selected_x_axis, selected_size):
    # Filter data based on selected school level
    selected_data = MIGENEDCOMPLETEWITHECA[MIGENEDCOMPLETEWITHECA['School Level'] == selected_school_level]
    
    # Extract selected columns
    y = selected_data['per_ecodis']
    x = selected_data[selected_x_axis]
    size = selected_data[selected_size] * 4
    color = selected_data[selected_size]
    count = selected_data[selected_size]

    # Update x-axis label if it's one of the modified options
    x_axis_label = selected_x_axis
    if selected_x_axis in updated_labels:
        x_axis_label = updated_labels[selected_x_axis]

    # Define the title based on the selected outcome
    outcome_title = {
        'numextracurricular': 'Number of Extracurricular Activities',
        'numclubs': 'Number of Clubs',
        'numsports': 'Number of Sports'
    }
    
    # Create scatter plot
    fig = px.scatter(
        x=x, y=y, size=size, color=color, hover_data={'size': count},
        color_continuous_scale='RdYlBu', opacity=0.8,
        labels={'x': f'% {x_axis_label.replace("per_", "").title()}',
                'y': '% Economically Disadvantaged'},
        title=f'{outcome_title[selected_size]}: {selected_school_level}' 
    )
    
    # Update x-axis category order and labels
    return fig

# Run the application
if __name__ == '__main__':
    app.run_server(debug=True)